In [32]:
import torch.nn as nn
from transformers import AutoTokenizer, ModernBertModel
import torch.nn.functional as F

class ModernBertClassifier(nn.Module):
    def __init__(self, num_labels, model_name):
        super().__init__()
        self.bert = ModernBertModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Linear(768, 256),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.classifier(outputs.last_hidden_state[:, 0, :])

In [33]:
import torch
import dill
import torch.nn.functional as F
from transformers import AutoTokenizer

model_quantized = torch.load(r'C:\Users\Kulde\OneDrive\Desktop\OFFICE\Emotion_Sentiment_Analysis\models\CLEANED\model_dill_2_layer_Cleandata.pth', pickle_module=dill,map_location=torch.device('cpu'))
# model = torch.load(r'C:\Users\Kulde\OneDrive\Desktop\OFFICE\Emotion_Sentiment_Analysis\models\model_experiment_2\modernbert_sentiment_model.pth', weights_only=False, map_location=torch.device('cpu'))
# tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

In [34]:
model_quantized

OptimizedModule(
  (_orig_mod): ModernBertClassifier(
    (bert): ModernBertModel(
      (embeddings): ModernBertEmbeddings(
        (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
        (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (layers): ModuleList(
        (0): ModernBertEncoderLayer(
          (attn_norm): Identity()
          (attn): ModernBertAttention(
            (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
            (rotary_emb): ModernBertRotaryEmbedding()
            (Wo): Linear(in_features=768, out_features=768, bias=False)
            (out_drop): Identity()
          )
          (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): ModernBertMLP(
            (Wi): Linear(in_features=768, out_features=2304, bias=False)
            (act): GELUActivation()
            (drop): Dropout(p=0.0, inplace=False)
            (Wo): Linear(in_fea

In [22]:
print("Model loaded:")
# print(model)


Model loaded:


In [ ]:
from pypdf import PdfReader

# Load the PDF file
reader = PdfReader(r"C:\Users\Kulde\OneDrive\Desktop\RESUME\KULDIP_PANCHAL.pdf")

# Extract text from all pages
for page in reader.pages:
    print(page.extract_text())

In [49]:
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

In [51]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True
TORCH_LOGS="+dynamo"  
TORCHDYNAMO_VERBOSE=1

text = """The room is quiet and orderly. The clock on the wall ticks steadily, marking the passage of time without hurry or delay. The curtains are open, allowing natural light to filter in, illuminating the space evenly. The air is neither warm nor cool, maintaining a comfortable temperature.

Outside, the weather is calm. The sky is a clear blue, with a few wisps of clouds drifting lazily. The trees stand tall and still, their leaves rustling gently in the occasional breeze. The street is neither busy nor deserted, with a steady flow of people going about their daily routines.

You sit at your desk, focused on the task at hand. The work is neither exciting nor boring, simply a series of steps to be completed methodically. You check items off your to-do list, one by one, without rush or procrastination. The environment is conducive to productivity, with minimal distractions.

Throughout the day, you interact with colleagues and clients, exchanging information and updates. The conversations are polite and professional, devoid of personal emotions or biases. Decisions are made based on facts and data, with a clear understanding of the goals and objectives.

Lunch is a simple affair, a balanced meal consumed at a reasonable pace. The cafeteria is neither crowded nor empty, with a hum of quiet conversation filling the air. You return to your desk, refreshed and ready to continue your work.

As the day progresses, you complete your tasks efficiently. There are no surprises or disruptions, just a steady flow of work. The office environment is neutral, with a focus on getting things done without unnecessary drama or excitement.

By the end of the day, you have accomplished what you set out to do. You leave the office, satisfied with your productivity. The commute home is uneventful, with traffic flowing smoothly. You arrive home, ready to relax and prepare for the next day.

The evening is spent in quiet reflection, planning for the future without dwelling on the past. You go to bed at a reasonable hour, ready to face whatever the next day brings with a calm and composed demeanor."""
text = re.sub(r'\s+',' ',text).strip().lower()
text = re.sub(r'[^a-z0-9\s]','',text)

inputs = tokenizer(text, padding=True, truncation=True, max_length=1500, return_tensors="pt")

with torch.no_grad():
    outputs = model_quantized(**inputs)
    predicted_class = torch.argmax(outputs, dim=1)

print("Predicted class:", predicted_class.item())

emotions = ['Aggresive','Fear','Happy','Neutral','Sad']
predicted_label = emotions[predicted_class]
predicted_label


Predicted class: 3


'Neutral'

In [18]:
print(outputs)

tensor([[ 4.7859, -4.8227,  1.6116,  0.0065, -4.0654]])


'Sad'

In [37]:
logits = outputs
# Apply softmax to get probabilities
probabilities = F.softmax(logits, dim=-1)
# Get the predicted class (index of the highest probability)
predicted_class = torch.argmax(probabilities, dim=-1).item()
# Get the predicted class label
predicted_label = model.config.id2label[predicted_class]
# Print the results
print(f"Input text: {text}")
print(f"Predicted class index: {predicted_class}")
print(f"Predicted class label: {predicted_label}")

AttributeError: 'ModernBertClassifier' object has no attribute 'config'

In [11]:
model.eval()

with torch.no_grad():
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

    # Example input text
    text = "I am so happy today!"
    # Tokenize the input text   
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=1500)
    # Move inputs to the same device as the model
    # inputs = {key: value.to(model.device) for key, value in inputs.items()}
    # Forward pass through the model
    outputs = model(**inputs)
    # Get the logits from the model output
    logits = outputs.logits
    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=-1)
    # Get the predicted class (index of the highest probability)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    # Get the predicted class label
    predicted_label = model.config.id2label[predicted_class]
    # Print the results
    print(f"Input text: {text}")
    print(f"Predicted class index: {predicted_class}")
    print(f"Predicted class label: {predicted_label}")

RuntimeError: RESUME instruction not found in code